In [ ]:
!apt install python3-libtorrent
import libtorrent as lt
import os
import shutil
import time
import datetime
from google.colab import files
from IPython.display import clear_output

In [85]:
if os.path.exists("/content/torrent_dir"):
  shutil.rmtree("/content/torrent_dir")

os.mkdir("/content/torrent_dir")
down_dir = "/content/torrent_dir"

In [ ]:
torrent_file = files.upload()
torrent_file_dir = os.path.abspath(list(torrent_file.keys())[0])

In [84]:
ses= lt.session()
ses.pause()
info = lt.torrent_info(torrent_file_dir)
ses.listen_on(6881, 6891)
params = {
    'ti':info,
    'save_path': '/content/torrent_dir',
    'storage_mode': lt.storage_mode_t(2),
    'paused': True,
    'auto_managed': True,
    'duplicate_is_error': True,
    'flat_merge_resume_trackers': True
    }

ses.pause()
handle = ses.add_torrent(params)

In [ ]:
for x in range(info.files().num_files()):
  full_path = info.files().file_path(x)
  print(full_path)

In [ ]:
folder_dict = {'top':[],'second':[],'third':[],'second_show':[]}
dict_key = ['top','second','third']
for x in range(info.files().num_files()):
  full_path = info.files().file_path(x)
  split_list = full_path.split('/')
  if len(full_path.split('/')) == 2:
    dict_key.pop()
  for i,j in enumerate(dict_key):
    nam = split_list[i]
    if nam not in folder_dict[j]:
      folder_dict[j].append(nam)

for num,i in enumerate(folder_dict['second']):
  sp = i.split('.')
  try:  
    if int(sp[0]) in list(range(0,10)):
      folder_dict['second_show'].append('0'+i)
    else:
      folder_dict['second_show'].append(i)

  except:
    folder_dict['second_show'].append(i)

for name in sorted(folder_dict['top']):
  print(name,"\n*****************************")
for i,name in enumerate(sorted(folder_dict['second_show'])):
  print(f'{i:3}',' ',name)

In [ ]:
download_list = []
num_files = info.files().num_files()
print(num_files)
prios = [0] * num_files
while True:
  x = input("please input index to download. Input 'all' to download all, \n'del' remove input and 'break' to finish!\nYour input:  ")
  try:
    if int(x) <= num_files:
      download_list.append(int(x))
      print(download_list)
    
    else:
      print("please input a valid index.")

  except:
    if x == "all":
      download_list = [7] * num_files
      break
    elif x == "break":
      print("break!")
      break
    elif x == "del":
      print("deleted last input!")
      download_list.pop()
      print(download_list)
    else:
      print("please type a valid input.")

print_list = ['❌']*len(folder_dict['second_show'])
for i in download_list:
  print_list[i] = "✅"

for name in sorted(folder_dict['top']):
  print("\n\nDonwload_list check\n\n*********************************\n")
for i,name in enumerate(sorted(folder_dict['second_show'])):
  print(f'{print_list[i]}{i:3}',' ',name)

In [ ]:
sorted_idx = []
num_prev = 0
KEEPON = True
while KEEPON:
  for i in sorted(folder_dict['second']):
    if type(list(i)[0]) != type(1):
      print("file names not start with digit!")
      sorted_idx = folder_dict['second']
      KEEPON = False
      break    
    num = int(i.split(".")[0])
    if not sorted_idx:
      sorted_idx.append(i)
    elif num == 1 + int(sorted_idx[-1].split('.')[0]):
      sorted_idx.append(i)
    elif len(sorted_idx) == len(folder_dict['second']):
      KEEPON = False
      break 
    else: pass
  
for i in sorted_idx:
  print(i)


In [ ]:
download_names = []
download_index = []
for i in download_list:
  download_names.append(sorted_idx[i])

for index,x in enumerate(range(info.files().num_files())):
  full_path = info.files().file_path(x)
  for name in download_names:
    if name in full_path.split('/'):
      print(full_path)
      download_index.append(index)

print("\n\nAbove files are about to be downloaded...")

In [ ]:
for i in download_index:
  prios[i] = 7
handle.prioritize_files(prios)
handle.prioritize_pieces(prios)

ses.resume()
handle.resume()

ses.start_dht()
print("download started @ ",datetime.datetime.now())
print("Downloading : ",handle.name())

while handle.status().state != lt.torrent_status.seeding:
  s= handle.status()
  state_str = ['queued', 'checking', 'downloading metadata', 
              'downloading', 'finished', 'seeding', 'allocating']
  clear_output(wait=True)
  print ('%.4f%% complete (down: %.1f kb/s up: %.1f kB/s seeds:%d peers: %d) %s ' %(s.progress * 100,
          s.download_rate / 1000, s.upload_rate / 1000, s.num_seeds, s.num_peers, state_str[s.state]))
  time.sleep(0.1)
  if state_str[s.state] == "finished":
    break
print("download completed!")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')